# Quick report
Ließt Daten, die ähnlich der Datei *valuesncprogram.json* vorliegen, in ein Pandas-Dataframe ein. demo von *pandas_profiling*, das einen report über das eingegebene DataFrame ausgibt.

## TODO
* _values.json_ enthält größere verschachtelte objekte. Wie wichtig sind diese für unseren Machine Learning-Prozess?
* Generell abklären: welche daten werden tatsächlich benötigt? reduzieren der Daten auf die wichtigsten Spalten

In [1]:
import pandas as pd
import pandas_profiling

def readMachineJSON(path, dataLike="valuesncprogram"):
    df = pd.read_json(filename,orient="records", lines=True)
    if dataLike == "valuesncprogram":
        df["_id"] = df["_id"].apply(lambda x: x["$oid"])
        df["timeStamp"] = df["timeStamp"].apply(lambda x: pd.to_datetime(x["$date"]))
        df["timeStamp"] = pd.to_datetime(df["timeStamp"])
    elif dataLike == "values":
        df["_id"] = df["_id"].apply(lambda x: x["$oid"])
        #todo machineStatusList auspacken: [{"_id":"SENSORDATA.12430012063.Channel1.Common_Machine_Data.Status_Signal_Lamp.STATUS","ValueUnit":"","Value":"false","Timestamp":{"$date":"2019-08-02T09:11:34.489Z"},"StatusId":"Signal_Lamp_RED","StatusType":"RED"},{"_id":"SENSORDATA.12430012063.Channel1.Common_Machine_Data.Status_Signal_Lamp.STATUS","ValueUnit":"","Value":"false","Timestamp":{"$date":"2019-08-02T09:11:34.534Z"},"StatusId":"Signal_Lamp_YELLOW","StatusType":"YELLOW"}]
    return df

In [32]:
filename = "valuesncprogram.json"
df = readMachineJSON(filename)
print(df.shape)
print(df.dtypes)

NameError: name 'readMachineJSON' is not defined

In [50]:
import pandas as pd

df = pd.read_json("values_mdb-export.json",orient="records", lines=True)

In [51]:
import ast

df["timeStamp"] = df["timeStamp"].apply(lambda x: x["$date"]["$numberLong"])
df["timeStampMqttClient"] = df["timeStampMqttClient"].apply(lambda x: x["$date"]["$numberLong"])
df["_id"] = df["_id"].apply(lambda x: x["$oid"])
df["valueStatus"] = df["valueStatus"].apply(lambda x: x["$numberInt"])
df["value_number"] = df["value_number"].apply(lambda x: list(x.values())[0])
df.head()

,Error,ValueID,ValueUnit,_id,channelId,componentID,machineID,machineStatusList,statusID,statusIsMachine,statusTyp,timeStamp,timeStampMqttClient,value,valueStatus,value_number
0,0,12430012063.Main_Spindle.Actual_Power,%,5dc516968e15b329dc1341f3,Channel1,12430012063.Main_Spindle,12430012063,NaN,0,False,GREEN,1573197461887,1573197437715,0.0,0,0
1,0,12430012063.Main_Spindle.Actual_Speed_Rate,RPM,5dc516968e15b329dc1341f1,Channel1,12430012063.Main_Spindle,12430012063,NaN,0,False,GREEN,1573197461887,1573197437717,0.0,0,0
2,0,12430012063.Main_Spindle.Actual_Position_MCS,Grad,5dc516968e15b329dc1341f2,Channel1,12430012063.Main_Spindle,12430012063,NaN,0,False,GREEN,1573197461888,1573197437728,360.0,0,360
3,0,12430012063.Main_Spindle.MPC_Veff_Total,mm/s,5dc516968e15b329dc1341f5,Channel1,12430012063.Main_Spindle,12430012063,NaN,0,False,GREEN,1573197461890,1573197437715,0.0,0,0
4,0,12430012063.C1_Axis.Actual_Power,%,5dc516968e15b329dc1341ec,Channel1,12430012063.C1_Axis,12430012063,NaN,0,False,GREEN,1573197461891,1573197437715,0.0,0,0
